# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up
In terminal, import the data from establishments.json by running the following command: `mongoimport --db uk_food --collection establishments --file resources/establishments.json --jsonArray`

In [1]:
from pymongo import MongoClient
from pprint import pprint

In [2]:
mongo = MongoClient(port=27017)

In [3]:
mongo.list_database_names()

['admin', 'config', 'local', 'uk_food']

In [4]:
uk_food_db = mongo["uk_food"]

In [5]:
uk_food_db.list_collection_names()


['establishments']

In [6]:
pprint(uk_food_db["establishments"].find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('678066cfdaeae20bf64beee9'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [7]:
uk_food_establishments = uk_food_db["establishments"]